<a href="https://www.kaggle.com/code/kusumakar/spam-ham-prediction-gaussian-nb-98-35-accuracy?scriptVersionId=199833625" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-sms-classification-using-nlp/Spam_SMS.csv


### LOAD THE DATA

In [2]:
df=pd.read_csv("/kaggle/input/spam-sms-classification-using-nlp/Spam_SMS.csv")

### VIEW THE DATA

In [3]:
df.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
import nltk

### IMPORT LIBRARIES

In [5]:
from nltk import word_tokenize, pos_tag,ne_chunk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import string



#### TEXT PREPROCESSING

In [6]:
cleaned=[]
texts = df.Message.to_list()
for i in range(len(texts)):
    text=texts[i]
    text = text.lower()

    # 2. Remove URLs and emails
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # Remove email addresses

    # 3. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 4. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # 6. Tokenize the text
    words = word_tokenize(text)

    # 7. Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # 8. Lemmatization
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    # 9. Handle contractions (Optional)
    
    cleaned_text = ' '.join(stemmed_words)
    cleaned.append(cleaned_text)
    

### CONVERT THE TEXT INTO BAG OF WORDS USING COUNTVECTORIZER CLASS

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features=2500)
X=cv.fit_transform(cleaned).toarray()

In [8]:
Y=pd.get_dummies(df['Class'])

In [9]:
Y=Y['spam']

### SPLIT THE DATA TO TRAIN AND TEST

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25)

### TRAIN THE MODEL

In [11]:
from sklearn.naive_bayes import MultinomialNB
model =MultinomialNB()
model.fit(X_train,Y_train)

MultinomialNB()

In [12]:
y_pred = model.predict(X_test)

### PERFORMANCE EVALUATION OF THE MODEL

In [13]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy : ",round(accuracy_score(y_pred,Y_test),4)*100,"%")

Accuracy :  98.21 %


### CLASSIFICATION REPORT

In [14]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99      1213
        True       0.93      0.93      0.93       181

    accuracy                           0.98      1394
   macro avg       0.96      0.96      0.96      1394
weighted avg       0.98      0.98      0.98      1394

